# **Tution cost of attending the public colleges**


In [2]:
pip install snowflake-connector-python==2.3.6 > /dev/null

In [3]:
#read required information to connect to Snowflake database
from getpass import getpass
accountName = getpass('Enter Snowflake account name:')
userName = getpass('Enter Snowflake user name:')
password = getpass('Enter Snowflake password:')

databaseName = 'ipeds' #@param {type:"string"}
sourceSchema='Public'#@param {type:"string"} 
warehouseName = 'COMPUTE_WH'#@param{type:"string"}


Enter Snowflake account name:··········
Enter Snowflake user name:··········
Enter Snowflake password:··········


In [41]:
# Source System to gather Statistics from
import snowflake.connector
import pandas as pd
from __future__ import print_function
from ipywidgets import interactive

import ipywidgets as widgets

# Gets the version
ctx = snowflake.connector.connect(
    user=userName,
    password=password,
    account=accountName,
    database=databaseName,
    warehouse=warehouseName    
    )
cs = ctx.cursor()

try:
  sql = """
WITH CTE_Tution AS
            (
            SELECT   A.ACADEMICYEAR  
                    ,A.INSTITUTIONIDENTIFIER AS ID
                    ,B.INSTITUTIONNAME AS NAME
                    ,B.STATE
                    ,SUM(A.INSTATEAVERAGETUITIONFULLTIMEUNDERGRADUATES) AS InStateTution
                    ,SUM(A.OUTOFSTATEAVERAGETUITIONFULLTIMEUNDERGRADUATES) AS OutOfStateTution
                    ,ROW_NUMBER() OVER (PARTITION BY A.ACADEMICYEAR ORDER BY InStateTution DESC) AS MaxFeeCounter
            FROM INSTITUTIONALCHARGE A
              INNER JOIN ACADEMICINSTITUTION B ON A.INSTITUTIONIDENTIFIER = B.INSTITUTIONIDENTIFIER
              INNER JOIN INSTITUTIONCONTROL ICTR ON ICTR.CODE = B.CONTROLOFINSTITUTION 
              WHERE UPPER(ICTR.DESCRIPTION) LIKE '%PUBLIC%' AND B.DEGREEGRANTINGSTATUS = 1  
            GROUP BY A.ACADEMICYEAR,ID,NAME,B.STATE
          )
          SELECT ACADEMICYEAR,
                 ID AS "Institution ID",
                 NAME "Instituon Name",
                 STATE "State Code",
                 InStateTution AS "In State Tution($)",
                 OutOfStateTution "Out Of State Tution($)" ,
                 MaxFeeCounter "in-state-tution_index"
          FROM CTE_Tution ;
        """
  #print (sql)
  cs.execute(sql)
  records = cs.fetchall()
  names = [x[0] for x in cs.description]
  df1 = pd.DataFrame(records,columns = names)
finally:
    cs.close()
    ctx.close()

In [42]:
  # Select the year for report  
  def f(Year):
    return Year  
  w = interactive(f,Year = df1.ACADEMICYEAR .unique());
  display(w);

interactive(children=(Dropdown(description='Year', options=(2017, 2018, 2019), value=2017), Output()), _dom_cl…

In [43]:
#Select number of institute to be displayed 
slider=widgets.IntSlider(
    min=10,
    max=40,
    step=2,
    description='Number of Institutiuons to be displayed on the report ',
    value=3
)
display(slider)

IntSlider(value=10, description='Number of Institutiuons to be displayed on the report ', max=40, min=10, step…

In [44]:
# Add formatting to dataset
import warnings
warnings.filterwarnings('ignore')
dfFilter = df1[df1.ACADEMICYEAR.eq(w.result)]  
pd.options.display.float_format = '${:,.2f}'.format 
dfFilter['In State Tution($)'] = dfFilter['In State Tution($)'].astype(float)
dfFilter['Out Of State Tution($)'] = dfFilter['Out Of State Tution($)'].astype(float)
#print(dfFilter)

In [45]:
# Apply fiter and display the results
topn=slider.value 
dfFilter=dfFilter.nlargest(topn,'In State Tution($)')
dfFilter=dfFilter.drop(columns=['in-state-tution_index'])
dfFilter = dfFilter.rename(columns = {"ACADEMICYEAR":"Academic Year"}) 
from IPython.display import HTML
HTML(dfFilter.to_html(index=False))

Academic Year,Institution ID,Instituon Name,State Code,In State Tution($),Out Of State Tution($)
2018,231837,Radford University-Carilion,VA,"$26,416.00","$26,416.00"
2018,215293,University of Pittsburgh-Pittsburgh Campus,PA,"$18,130.00","$31,102.00"
2018,214777,Pennsylvania State University-Main Campus,PA,"$17,416.00","$33,820.00"
2018,126775,Colorado School of Mines,CO,"$16,650.00","$36,270.00"
2018,171128,Michigan Technological University,MI,"$16,500.00","$35,142.00"
2018,216339,Temple University,PA,"$16,080.00","$28,176.00"
2018,231174,University of Vermont,VT,"$15,936.00","$40,176.00"
2018,170976,University of Michigan-Ann Arbor,MI,"$15,897.00","$50,754.00"
2018,231624,William & Mary,VA,"$15,864.00","$38,735.00"
2018,218335,Medical University of South Carolina,SC,"$15,622.00","$23,340.00"
